In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline


In [2]:
words = open('/content/name.txt','r').read().splitlines()

In [3]:
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
len(words)

32033

In [5]:
# build the vocabulary of characters and mapping to from integers

cahrs = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(cahrs)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [7]:
# build the dataset

block_size = 3 # context length: how many characters do we take to
X,Y = [],[]
for w in words[:5]:
  print(w)
  context = [0]*block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    print(''.join(itos[i] for i in context), '----->',itos[ix])
    context = context[1:] + [ix]
X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... -----> e
..e -----> m
.em -----> m
emm -----> a
mma -----> .
olivia
... -----> o
..o -----> l
.ol -----> i
oli -----> v
liv -----> i
ivi -----> a
via -----> .
ava
... -----> a
..a -----> v
.av -----> a
ava -----> .
isabella
... -----> i
..i -----> s
.is -----> a
isa -----> b
sab -----> e
abe -----> l
bel -----> l
ell -----> a
lla -----> .
sophia
... -----> s
..s -----> o
.so -----> p
sop -----> h
oph -----> i
phi -----> a
hia -----> .


In [8]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [9]:
X

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1],
        [ 0,  0,  0],
        [ 0,  0, 15],
        [ 0, 15, 12],
        [15, 12,  9],
        [12,  9, 22],
        [ 9, 22,  9],
        [22,  9,  1],
        [ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1, 22],
        [ 1, 22,  1],
        [ 0,  0,  0],
        [ 0,  0,  9],
        [ 0,  9, 19],
        [ 9, 19,  1],
        [19,  1,  2],
        [ 1,  2,  5],
        [ 2,  5, 12],
        [ 5, 12, 12],
        [12, 12,  1],
        [ 0,  0,  0],
        [ 0,  0, 19],
        [ 0, 19, 15],
        [19, 15, 16],
        [15, 16,  8],
        [16,  8,  9],
        [ 8,  9,  1]])

In [10]:
Y

tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
         1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0])

First we need to make the lookup table C

In [11]:
C = torch.randn((27,2))

In [12]:
C

tensor([[-0.8292, -1.3097],
        [ 0.1081,  0.7649],
        [-0.6037,  0.4702],
        [ 0.2986, -0.3646],
        [ 1.3390, -1.2415],
        [-0.5771, -0.8635],
        [-1.0819, -0.1001],
        [-0.3676, -0.2304],
        [-0.9150, -1.0633],
        [ 1.4763,  1.2915],
        [ 0.1600, -0.5231],
        [-1.0658,  0.3182],
        [ 1.0499, -2.1524],
        [ 0.8836, -0.9002],
        [ 0.3854, -1.7286],
        [ 0.9190, -0.6958],
        [ 0.7513, -1.3718],
        [ 0.2336, -0.7989],
        [-0.1675,  0.7125],
        [ 1.5124,  1.6592],
        [-1.1942,  0.8439],
        [-1.1364, -0.4116],
        [-0.0609, -0.4734],
        [-0.4757, -0.0656],
        [ 0.0188, -0.3433],
        [ 0.4467, -1.4975],
        [-0.8613,  1.1699]])

In [13]:
C[5]

tensor([-0.5771, -0.8635])

In [17]:
F.one_hot(torch.tensor(5),num_classes=27).float() @ C


tensor([-0.5771, -0.8635])

In [18]:
C[5]

tensor([-0.5771, -0.8635])

In [19]:
C[[5,6,7]]

tensor([[-0.5771, -0.8635],
        [-1.0819, -0.1001],
        [-0.3676, -0.2304]])

In [22]:
C[X]

tensor([[[-0.8292, -1.3097],
         [-0.8292, -1.3097],
         [-0.8292, -1.3097]],

        [[-0.8292, -1.3097],
         [-0.8292, -1.3097],
         [-0.5771, -0.8635]],

        [[-0.8292, -1.3097],
         [-0.5771, -0.8635],
         [ 0.8836, -0.9002]],

        [[-0.5771, -0.8635],
         [ 0.8836, -0.9002],
         [ 0.8836, -0.9002]],

        [[ 0.8836, -0.9002],
         [ 0.8836, -0.9002],
         [ 0.1081,  0.7649]],

        [[-0.8292, -1.3097],
         [-0.8292, -1.3097],
         [-0.8292, -1.3097]],

        [[-0.8292, -1.3097],
         [-0.8292, -1.3097],
         [ 0.9190, -0.6958]],

        [[-0.8292, -1.3097],
         [ 0.9190, -0.6958],
         [ 1.0499, -2.1524]],

        [[ 0.9190, -0.6958],
         [ 1.0499, -2.1524],
         [ 1.4763,  1.2915]],

        [[ 1.0499, -2.1524],
         [ 1.4763,  1.2915],
         [-0.0609, -0.4734]],

        [[ 1.4763,  1.2915],
         [-0.0609, -0.4734],
         [ 1.4763,  1.2915]],

        [[-0.0609, -0

In [23]:
C[X].shape

torch.Size([32, 3, 2])

In [24]:
X[13,2]

tensor(1)

In [25]:
C[X][13,2]

tensor([0.1081, 0.7649])

In [26]:
C[1]

tensor([0.1081, 0.7649])

In [27]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [28]:
W1 = torch.rand((6,100))
b1 = torch.rand(100)


In [29]:
emb @ W1 + b1

RuntimeError: mat1 and mat2 shapes cannot be multiplied (96x2 and 6x100)

In [34]:
torch.cat([emb[:,0,:], emb[:,1,:], emb[:,2,:]],1).shape

torch.Size([32, 6])

In [35]:
torch.cat(torch.unbind(emb,1),1).shape

torch.Size([32, 6])

In [36]:
a = torch.arange(18)

In [37]:
a

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17])

As long as the multiplipcation of the number is the same, it will work

In [38]:
a.view(2,9)

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13, 14, 15, 16, 17]])

In [39]:
a.view(3,3,2)

tensor([[[ 0,  1],
         [ 2,  3],
         [ 4,  5]],

        [[ 6,  7],
         [ 8,  9],
         [10, 11]],

        [[12, 13],
         [14, 15],
         [16, 17]]])